In [1]:
import pandas as pd
import numpy as np
import warnings
import os
import sys
import urllib.request
import joblib
import simplejson as json
from time import time
warnings.filterwarnings(action='ignore')

import seaborn as sns
sns.set_palette("pastel")
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
%matplotlib inline

import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')

In [2]:
LPD_201902 = pd.read_csv('LOCAL_PEOPLE_DONG_201902.csv',index_col=False)
dfcolumns = pd.read_csv('LOCAL_PEOPLE_DONG_201903.csv',
                        nrows = 1)
LPD_201903 = pd.read_csv('LOCAL_PEOPLE_DONG_201903.csv', header = None, 
                         skiprows = 1, usecols = list(range(len(dfcolumns.columns))), 
                         names = dfcolumns.columns)
LPD_201904 = pd.read_csv('LOCAL_PEOPLE_DONG_201904.csv',index_col=False)
LPD_201905 = pd.read_csv('LOCAL_PEOPLE_DONG_201905.csv',index_col=False)
LPD_202002 = pd.read_csv('LOCAL_PEOPLE_DONG_202002.csv',index_col=False)
LPD_202003 = pd.read_csv('LOCAL_PEOPLE_DONG_202003.csv',index_col=False)
LPD_202004 = pd.read_csv('LOCAL_PEOPLE_DONG_202004.csv',index_col=False)
LPD_202005 = pd.read_csv('LOCAL_PEOPLE_DONG_202005.csv',index_col=False)

In [3]:
df = pd.concat([LPD_201902, LPD_201903, LPD_201904, LPD_201905, LPD_202002, LPD_202003, LPD_202004, LPD_202005])

In [4]:
df.rename(columns = {'남자0세부터9세생활인구수':'남자_0009세생활인구수'},inplace=True)
df.rename(columns = {'남자70세이상생활인구수':'남자_70세이상생활인구수'},inplace=True)
df.rename(columns = {'여자0세부터9세생활인구수':'여자_0009세생활인구수'},inplace=True)
df.rename(columns = {'여자70세이상생활인구수':'여자_70세이상생활인구수'},inplace=True)

In [6]:
for i in np.arange(1,13):
    temp = list(df.columns[4:])[i][:2]+'_'+list(df.columns[4:])[i][2:4]+list(df.columns[4:])[i][7:]
    name = list(df.columns[4:])[i]
    df.rename(columns={name:temp},inplace=True)

for i in np.arange(15,27):
    temp = list(df.columns[4:])[i][:2]+'_'+list(df.columns[4:])[i][2:4]+list(df.columns[4:])[i][7:]
    name = list(df.columns[4:])[i]
    df.rename(columns={name:temp},inplace=True)

In [9]:
cd = pd.read_csv('행정동코드_매핑정보.csv',encoding='cp949',header=1)
cd['G_DNG_NM'] = cd[['DO_NM','CT_NM','H_DNG_NM']].apply(lambda x: ' '.join(x), axis=1)
GU_DONG = cd[['H_DNG_CD','G_DNG_NM']]
GU_DONG.rename(columns = {'H_DNG_CD':'행정동코드'},inplace=True)

In [11]:
life_pup = pd.merge(df,GU_DONG,on='행정동코드',how='inner')

In [27]:
life_pup = pd.concat([life_pup[life_pup['G_DNG_NM'].str.contains('노원구')], life_pup[life_pup['G_DNG_NM'].str.contains('중구')]])

In [28]:
life_pup.rename(columns = {'기준일ID':'STD_DD'},inplace=True)

In [29]:
life_pup['STD_DD'] = life_pup['STD_DD'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

In [32]:
life_pup = life_pup.reset_index(drop=True)

In [33]:
life_pup.to_csv('local_people_data.csv',index=False)